# Lab 2.4 - Use your own data from a client app (python)

In this lab we'll be expading on the latest one and using our own data when calling the Azure OpenAI API.

For reference, the official docs are [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/use-your-data-quickstart?tabs=command-line%2Cpython-new&pivots=programming-language-python).

## Ensure required software is installed

We'll be using the same libraries as the previous lab. These should have been installed already as they are in requirements.txt

* `openai` - the OpenAI python library
* `python-dotenv` - to load environment variables from a .env file 

## Load the environment variables

We recommend creating a .env file with the following variables defined. .env files are ignored when pushing code to GitHub and are a good way to keep secrets out of your code.

Also, we'll be reusing these variables in other notebooks, so it's a good idea to keep them in a single place.

To use our own data, which is indexed in Azure AI Search, we need to provide the required environment variables.

In [ ]:
# Load the environment variables with dotenv
from dotenv import load_dotenv
load_dotenv()
import os

# Endpoint and API key can be found in Azure AI Studio -> Project Settings -> Project Properties -> Get API endpoints
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
# The name of the deployment to tbe used. Found in Azure AI Studio -> Deployments
AZURE_OPENAI_DEPLOYMENT_ID = os.getenv("AZURE_OPENAI_DEPLOYMENT_ID")

# Azure AI Search
AZURE_AI_SEARCH_ENDPOINT = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
AZURE_AI_SEARCH_API_KEY = os.getenv("AZURE_AI_SEARCH_API_KEY")
AZURE_AI_SEARCH_INDEX = os.getenv("AZURE_AI_SEARCH_INDEX")

# Print the endpoint to verify it was loaded correctly
print('If you see some text below, the endpoint was loaded successfully.')
print(AZURE_OPENAI_ENDPOINT)
print(f'Using model {AZURE_OPENAI_DEPLOYMENT_ID}')

In [ ]:
import openai

# Create the client 
client = openai.AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-02-01",
)

In [ ]:
# For convenience, use deployment as var instead of env var name
deployment = AZURE_OPENAI_DEPLOYMENT_ID

# By default, the OpenAI service does not know about our health plans
completion = client.chat.completions.create(
    model=deployment,
    messages=[
        {
            "role": "user",
            "content": "Which health plans are available in my organization (Contoso)?",
        },
    ]
)

# Get the response
print(completion.choices[0].message.content)

There are many ways to search our data and use it with the OpenAI service (eg: application logic, prompt flow, ...). 

Notice, however, we can pass the datastore as part of the extra_body parameter in the completion method. This is the most direct way to use our data.

In [ ]:
# Let's instruct the OpenAI service to query the data source where our data is hosted
completion = client.chat.completions.create(
    model=deployment,
    messages=[
        {
            "role": "user",
            "content": "Which health plans are available in Contoso?",
        },
    ],
    extra_body={
        "data_sources":[
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": AZURE_AI_SEARCH_ENDPOINT,
                    "index_name": AZURE_AI_SEARCH_INDEX,
                    "authentication": {
                        "type": "api_key",
                        "key": AZURE_AI_SEARCH_API_KEY,
                    }
                }
            }
        ],
    }
)

# Get the response
print(completion.choices[0].message.content)